# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# import csv from WeatherPy 
weather_csv = "../WeatherPy/output_data/cities.csv"

weather_df = pd.read_csv(weather_csv)

weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hearst,49.6835,-83.6665,3.45,96,100,7.36,CA,1644201140
1,1,Atuona,-9.8000,-139.0333,78.80,77,12,17.09,PF,1644201272
2,2,Albany,42.6001,-73.9662,21.31,76,10,4.18,US,1644201074
3,3,Carnarvon,-24.8667,113.6333,78.87,36,6,21.85,AU,1644201272
4,4,Hermanus,-34.4187,19.2345,65.80,92,66,4.25,ZA,1644201273
...,...,...,...,...,...,...,...,...,...,...
552,552,Colares,38.7992,-9.4469,52.79,51,4,5.01,PT,1644201459
553,553,Zhaoqing,23.0512,112.4597,59.67,91,100,3.44,CN,1644201459
554,554,Vilhena,-12.7406,-60.1458,69.28,98,100,3.20,BR,1644201460
555,555,Kaeo,-35.1000,173.7833,77.32,97,99,6.44,NZ,1644201460


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Config gmaps with api key.
gmaps.configure(api_key=g_key)

# Drop NA
weather_df = weather_df.dropna()

# Convert to Float
humidity = weather_df["Humidity"].astype(float)

# Storage 
location = weather_df[["Lat", "Lng"]].astype(float)

# Humidity Heatmap Layer
figure = gmaps.figure()

heat = gmaps.heatmap_layer(location, weights = humidity, dissipating = False, max_intensity = max(weather_df["Humidity"]), point_radius = 2)

figure.add_layer(heat)

figure

Figure(layout=FigureLayout(height='420px'))

In [8]:
# New DataFrame
ideal_weather_df = weather_df

"""
Conditionals: 

A max temperature lower than 80 degrees but higher than 70.
Wind speed less than 10 mph.
Zero cloudiness.

Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
"""
# A max temperature lower than 80 degrees but higher than 70.
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] < 80) & (ideal_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

# Drop NA
ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,79,Igboho,8.8333,3.7500,73.24,19,0,1.05,NG,1644201296
130,130,Springbok,-29.6643,17.8865,70.47,46,0,4.05,ZA,1644201314
135,135,New Norfolk,-42.7826,147.0587,77.88,30,0,3.00,AU,1644201218
139,139,Yaan,7.3833,8.5667,74.50,18,0,4.05,NG,1644201222
179,179,Devonport,-41.1667,146.3500,73.09,55,0,8.77,AU,1644201329
209,209,La Palma,33.8464,-118.0467,74.17,38,0,0.00,US,1644201139
217,217,Pedras de Fogo,-7.4019,-35.1164,77.63,77,0,3.60,BR,1644201342
310,310,Salalah,17.0151,54.0924,73.49,27,0,6.91,OM,1644201117
318,318,Ouro Sogui,15.6064,-13.3211,74.71,10,0,8.97,SN,1644201375
329,329,Nioro,13.3500,-15.7500,73.80,34,0,2.44,GM,1644201380


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_map_df = ideal_weather_df

# params, iteration, etc.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# iteration 
for index, row in hotel_map_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Result for the Index {index}: {row['City']}.")
    
    # response
    response = requests.get(url, params=params).json()
    
    results = response["results"]
    
    try:
        print(f"Nearest hotel is {results[0]['name']}.")
        hotel_map_df.loc[index, "Name of Hotel"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Skip Over Missing Info...")
        
    # print delimiter 
    print("---------------------------------")
# 10 results 

Result for the Index 79: Igboho.
Nearest hotel is More Hotet LTD.
---------------------------------
Result for the Index 130: Springbok.
Nearest hotel is Undulata Country Lodge.
---------------------------------
Result for the Index 135: New Norfolk.
Nearest hotel is The Shingles Riverside Cottages.
---------------------------------
Result for the Index 139: Yaan.
Skip Over Missing Info...
---------------------------------
Result for the Index 179: Devonport.
Nearest hotel is Gateway Hotel by Nightcap Plus.
---------------------------------
Result for the Index 209: La Palma.
Nearest hotel is La Quinta Inn & Suites by Wyndham Buena Park.
---------------------------------
Result for the Index 217: Pedras de Fogo.
Nearest hotel is Dormitório Pedra Do Sol.
---------------------------------
Result for the Index 310: Salalah.
Nearest hotel is HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel.
---------------------------------
Result for the Index 318: Ouro Sogui.
Nearest hotel is Hotel welma.
------

In [13]:
hotel_map_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Name of Hotel
79,79,Igboho,8.8333,3.7500,73.24,19,0,1.05,NG,1644201296,More Hotet LTD
130,130,Springbok,-29.6643,17.8865,70.47,46,0,4.05,ZA,1644201314,Undulata Country Lodge
135,135,New Norfolk,-42.7826,147.0587,77.88,30,0,3.00,AU,1644201218,The Shingles Riverside Cottages
139,139,Yaan,7.3833,8.5667,74.50,18,0,4.05,NG,1644201222,NaN
179,179,Devonport,-41.1667,146.3500,73.09,55,0,8.77,AU,1644201329,Gateway Hotel by Nightcap Plus
209,209,La Palma,33.8464,-118.0467,74.17,38,0,0.00,US,1644201139,La Quinta Inn & Suites by Wyndham Buena Park
217,217,Pedras de Fogo,-7.4019,-35.1164,77.63,77,0,3.60,BR,1644201342,Dormitório Pedra Do Sol
310,310,Salalah,17.0151,54.0924,73.49,27,0,6.91,OM,1644201117,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
318,318,Ouro Sogui,15.6064,-13.3211,74.71,10,0,8.97,SN,1644201375,Hotel welma
329,329,Nioro,13.3500,-15.7500,73.80,34,0,2.44,GM,1644201380,NaN


In [18]:
# Add marker layer ontop of heat map
locations = hotel_map_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name of Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_map_df.iterrows()]
locations = hotel_map_df[["Lat", "Lng"]]

# Symbol Layer
hotel_map_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', scale=2, info_box_content=hotel_info)

# Additional Layers
figure.add_layer(markers)
figure.add_layer(hotel_map_layer)

# Display figure

figure

Figure(layout=FigureLayout(height='420px'))